In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as K

2024-09-02 20:20:22.631204: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-02 20:20:22.637201: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-02 20:20:22.658911: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-02 20:20:22.692083: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-02 20:20:22.699706: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-02 20:20:22.720569: I tensorflow/core/platform/cpu_feature_gu

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
def create_dir(path):
    """Create a directory."""
    try:
        if not os.path.exists(path):
            os.makedirs(path)
    except FileExistsError as e:
        pass

In [3]:
physical_devices = tf.config.experimental.list_physical_devices("GPU")
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)


def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x

    return tf.numpy_function(f, [y_true, y_pred], tf.float32)


smooth = 1e-15


def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2.0 * intersection + smooth) / (
        tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth
    )


def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)


In [4]:
import tensorflow as tf
from tensorflow.keras.layers import (
    Activation,
    BatchNormalization,
    Concatenate,
    Conv2D,
    Conv2DTranspose,
    Input,
    MaxPool2D,
)
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Lambda, Dropout, MaxPooling2D, Conv2DTranspose, concatenate

physical_devices = tf.config.experimental.list_physical_devices("GPU")
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)


def build_unet(input_shape):

    inputs = Input(input_shape)

    lmda = Lambda(lambda x: x/255)(inputs)

    con_1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(lmda)
    con_1 = Dropout(0.1)(con_1)
    con_1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(con_1)
    pool_1 = MaxPooling2D((2, 2))(con_1)

    con_2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(pool_1)
    con_2 = Dropout(0.1)(con_2)
    con_2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(con_2)
    pool_2 = MaxPooling2D((2, 2))(con_2)

    con_3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(pool_2)
    con_3 = Dropout(0.2)(con_3)
    con_3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(con_3)
    pool_3 = MaxPooling2D((2, 2))(con_3)

    con_4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(pool_3)
    con_4 = Dropout(0.2)(con_4)
    con_4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(con_4)
    pool_4 = MaxPooling2D(pool_size=(2, 2))(con_4)

    con_5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(pool_4)
    con_5 = Dropout(0.3)(con_5)
    con_5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(con_5)

    # Expansive path
    u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(con_5)
    u6 = concatenate([u6, con_4])
    con_6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
    con_6 = Dropout(0.2)(con_6)
    con_6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(con_6)

    u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(con_6)
    u7 = concatenate([u7, con_3])
    con_7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
    con_7 = Dropout(0.2)(con_7)
    con_7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(con_7)

    u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(con_7)
    u8 = concatenate([u8, con_2])
    con_8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
    con_8 = Dropout(0.1)(con_8)
    con_8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(con_8)

    u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(con_8)
    u9 = concatenate([u9, con_1], axis=3)
    con_9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
    con_9 = Dropout(0.1)(con_9)
    con_9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(con_9)

    outputs = Conv2D(1, (1, 1), activation='sigmoid')(con_9)

    model = Model(inputs=[inputs], outputs=[outputs], name="U-Net")

    return model


In [ ]:
import os
from glob import glob

import cv2
import numpy as np
import tensorflow as tf
from sklearn.utils import shuffle
from tensorflow.keras.callbacks import (
    CSVLogger,
    EarlyStopping,
    ModelCheckpoint,
    ReduceLROnPlateau,
    TensorBoard,
)
from tensorflow.keras.metrics import Accuracy, Precision, Recall
from tensorflow.keras.optimizers import Adam

# from a_image_annotation import create_dir
# from metrics import dice_coef, dice_loss, iou
# from model_1 import build_unet

physical_devices = tf.config.experimental.list_physical_devices('GPU')
print(physical_devices)
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

W = 512
H = 512


def shuffling(x, y):
    x, y = shuffle(x, y, random_state=42)
    return x, y

def load_data(path):
    x = sorted(glob(os.path.join(path, "images", "*.jpg")))
    y = sorted(glob(os.path.join(path, "masks", "*.jpg")))
    return x, y

def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR + cv2.IMREAD_IGNORE_ORIENTATION)
    x = x/255.0
    x = x.astype(np.float32)
    return x

def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = x/255.0
    x = x > 0.5
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=-1)
    return x

def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([H, W, 3])
    y.set_shape([H, W, 1])
    return x, y

def tf_dataset(x, y, batch=8):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(10)
    return dataset


if __name__ == "__main__":
    """ Seeding """
    np.random.seed(42)
    tf.random.set_seed(42)

    """ Directory for storing files """
    save_model = "save_model_1"
    create_dir(save_model)

    """ Hyperparameters """
    batch_size = 15
    lr = 1e-4
    num_epochs = 50

    model_path = os.path.join(save_model, "model.keras")
    csv_path = os.path.join(save_model, "data.csv")

    """ Dataset """
    # dataset_path = os.path.join("data")
    train_path = os.path.join("/content/drive/MyDrive/Colab Notebooks/sem4/3_unet/aj_unet_code0", "data", "train_data", "512x512")
    valid_path = os.path.join("/content/drive/MyDrive/Colab Notebooks/sem4/3_unet/aj_unet_code0", "data", "val_data", "512x512")

    train_x, train_y = load_data(train_path)
    train_x, train_y = shuffling(train_x, train_y)
    valid_x, valid_y = load_data(valid_path)
    print(f"Train: {len(train_x)} - {len(train_y)}")
    print(f"Valid: {len(valid_x)} - {len(valid_y)}")

    train_dataset = tf_dataset(train_x, train_y, batch=batch_size)
    valid_dataset = tf_dataset(valid_x, valid_y, batch=batch_size)

    """ Model """
    model = build_unet((H, W, 3))
    model.summary()

    # model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    metrics = [dice_coef, Recall(), Precision(), Accuracy()]
    model.compile(loss=dice_loss, optimizer=Adam(lr), metrics=metrics)

    callbacks = [
        ModelCheckpoint(model_path, verbose=1, save_best_only=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, min_lr=1e-7, verbose=1),
        CSVLogger(csv_path),
        TensorBoard(),
        EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=False),
    ]

    model.fit(
        train_dataset,
        epochs=num_epochs,
        validation_data=valid_dataset,
        callbacks=callbacks,
        shuffle=False
    )

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Train: 786 - 786
Valid: 100 - 100


Model: "U-Net"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 512, 512, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lambda (Lambda)           │ (None, 512, 512, 3)    │              0 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 512, 512, 16)   │            448 │ lambda[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout (Dropout)         │ (None, 512, 512, 16)   │              0 │ conv2d[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 512, 512, 16)   │          2,320 │ dropout[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d             │ (None, 256, 256, 16)   │              0 │ conv2d_1[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2 (Conv2D)         │ (None, 256, 256, 32)   │          4,640 │ max_pooling2d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_1 (Dropout)       │ (None, 256, 256, 32)   │              0 │ conv2d_2[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_3 (Conv2D)         │ (None, 256, 256, 32)   │          9,248 │ dropout_1[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_1           │ (None, 128, 128, 32)   │              0 │ conv2d_3[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_4 (Conv2D)         │ (None, 128, 128, 64)   │         18,496 │ max_pooling2d_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_2 (Dropout)       │ (None, 128, 128, 64)   │              0 │ conv2d_4[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_5 (Conv2D)         │ (None, 128, 128, 64)   │         36,928 │ dropout_2[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_2           │ (None, 64, 64, 64)     │              0 │ conv2d_5[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_6 (Conv2D)         │ (None, 64, 64, 128)    │         73,856 │ max_pooling2d_2[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_3 (Dropout)       │ (None, 64, 64, 128)    │              0 │ conv2d_6[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_7 (Conv2D)         │ (None, 64, 64, 128)    │        147,584 │ dropout_3[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_3           │ (None, 32, 32, 128)    │              0 │ conv2d_7[0][0]         │
│ (MaxPooling2D)       

 Total params: 1,941,105 (7.40 MB)

 Trainable params: 1,941,105 (7.40 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 9s/step - accuracy: 0.0000e+00 - dice_coef: 0.0120 - loss: 0.9880 - precision: 0.0103 - recall: 0.8700
Epoch 1: val_loss improved from inf to 0.99150, saving model to save_model_1/model.keras
53/53 ━━━━━━━━━━━━━━━━━━━━ 687s 12s/step - accuracy: 0.0000e+00 - dice_coef: 0.0120 - loss: 0.9880 - precision: 0.0103 - recall: 0.8718 - val_accuracy: 0.0000e+00 - val_dice_coef: 0.0085 - val_loss: 0.9915 - val_precision: 0.0090 - val_recall: 0.9997 - learning_rate: 1.0000e-04
Epoch 2/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 449ms/step - accuracy: 0.0000e+00 - dice_coef: 0.0126 - loss: 0.9874 - precision: 0.0126 - recall: 1.0000
Epoch 2: val_loss improved from 0.99150 to 0.98841, saving model to save_model_1/model.keras
53/53 ━━━━━━━━━━━━━━━━━━━━ 27s 503ms/step - accuracy: 0.0000e+00 - dice_coef: 0.0126 - loss: 0.9874 - precision: 0.0126 - recall: 1.0000 - val_accuracy: 0.0000e+00 - val_dice_coef: 0.0116 - val_loss: 0.9884 - val_precision: 0.0083 - val_recall: 1.0

# d_eval_1

In [5]:
%pip install tqdm

In [7]:

import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

from glob import glob

import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    jaccard_score,
    precision_score,
    recall_score,
)
from tensorflow.keras.utils import CustomObjectScope
from tqdm import tqdm

physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

H = 512
W = 512

""" Creating a directory """
def create_dir(path):
    """Create a directory."""
    try:
        if not os.path.exists(path):
            os.makedirs(path)
    except FileExistsError as e:
        pass


def save_results(image, mask, y_pred, save_image_path):
    ## i - m - y
    line = np.ones((H, 10, 3)) * 128

    """ Mask """
    mask = np.expand_dims(mask, axis=-1)    ## (512, 512, 1)
    mask = np.concatenate([mask, mask, mask], axis=-1)  ## (512, 512, 3)

    """ Predicted Mask """
    y_pred = np.expand_dims(y_pred, axis=-1)    ## (512, 512, 1)
    y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1)  ## (512, 512, 3)
    y_pred = y_pred * 255

    cat_images = np.concatenate([image, line, mask, line, y_pred], axis=1)
    cv2.imwrite(save_image_path, cat_images)


def load_data(path):
    x = sorted(glob(os.path.join(path, "images", "*.jpg")))
    y = sorted(glob(os.path.join(path, "masks", "*.jpg")))
    return x, y




if __name__ == "__main__":
    """ Seeding """
    np.random.seed(42)
    tf.random.set_seed(42)

    """ Directory for storing files """
    # create_dir("results_1")
    save_image_path = os.path.join("/content/drive/MyDrive/Colab Notebooks/sem4/3_unet/aj_unet_code0", "results", "test_data", '512x512')
    create_dir(save_image_path)

    """ Loading model """
    with CustomObjectScope({'iou': iou, 'dice_coef': dice_coef, 'dice_loss': dice_loss}):
        model = tf.keras.models.load_model("save_model_1/model.keras", safe_mode=False)

    """ Load the dataset """
    test_path = os.path.join("/content/drive/MyDrive/Colab Notebooks/sem4/3_unet/aj_unet_code0", "data", "test_data", "512x512")
    # test_path = os.path.join("data", "test_data", "512x512")

    test_x, test_y = load_data(test_path)
    print(f"Test: {len(test_x)} - {len(test_y)}")

    """ Evaluation and Prediction """
    SCORE = []
    for x, y in tqdm(zip(test_x, test_y), total=len(test_x)):
        """ Extract the name """
        # save_image_path = os.path.join("result_final", "test_data", '512x512', f"{name}")
        name = x.split("/")[-1]

        """ Reading the image """
        image = cv2.imread(x, cv2.IMREAD_COLOR + cv2.IMREAD_IGNORE_ORIENTATION)
        x = image/255.0
        x = np.expand_dims(x, axis=0)

        """ Reading the mask """
        mask = cv2.imread(y, cv2.IMREAD_GRAYSCALE)
        y = mask/255.0
        y = y > 0.5
        y = y.astype(np.int32)

        """ Prediction """
        y_pred = model.predict(x)[0]
        y_pred = np.squeeze(y_pred, axis=-1)
        y_pred = y_pred > 0.5
        y_pred = y_pred.astype(np.int32)

        """ Saving the prediction """
        save_image_path = os.path.join("/content/drive/MyDrive/Colab Notebooks/sem4/3_unet/aj_unet_code0", "results", "test_data", '512x512', f'{name}')
        save_results(image, mask, y_pred, save_image_path)

        """ Flatten the array """
        y = y.flatten()
        y_pred = y_pred.flatten()

        """ Calculating the metrics values """
        acc_value = accuracy_score(y, y_pred)
        f1_value = f1_score(y, y_pred, labels=[0, 1], average="binary", zero_division=1)
        jac_value = jaccard_score(y, y_pred, labels=[0, 1], average="binary", zero_division=1)
        recall_value = recall_score(y, y_pred, labels=[0, 1], average="binary", zero_division=1)
        precision_value = precision_score(y, y_pred, labels=[0, 1], average="binary", zero_division=1)
        SCORE.append([name, acc_value, f1_value, jac_value, recall_value, precision_value])

    """ Metrics values """
    score = [s[1:]for s in SCORE]
    score = np.mean(score, axis=0)
    print(f"Accuracy: {score[0]:0.5f}")
    print(f"F1: {score[1]:0.5f}")
    print(f"Jaccard: {score[2]:0.5f}")
    print(f"Recall: {score[3]:0.5f}")
    print(f"Precision: {score[4]:0.5f}")

    df = pd.DataFrame(SCORE, columns=["Image", "Accuracy", "F1", "Jaccard", "Recall", "Precision"])
    df.to_csv("save_model_1/d_evel_test_score.csv")


Test: 94 - 94


  0%|          | 0/94 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step


  1%|          | 1/94 [00:07<10:54,  7.04s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step


 99%|█████████▉| 93/94 [02:57<00:01,  1.85s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step


100%|██████████| 94/94 [02:59<00:00,  1.91s/it]

Accuracy: 0.85170
F1: 0.05579
Jaccard: 0.01792
Recall: 0.41745
Precision: 0.01930
